In [3]:
import pandas as pd
from besos import eppy_funcs as ef
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem
import numpy as np
import copy
from seaborn import pairplot
import matplotlib.pyplot as plt

HD=0

## Parameters for E+ simulations

In [4]:
#here we're building the range within the optimization will happen

building_params= []
orientations = [180,270] #,270
WWRs = [.05, .15, .50] #
thickness = np.linspace(0.0001,0.35,10)
for orientation in orientations:
    for w in WWRs:
        for th in thickness:
            building_params.append({'Orientation': orientation,
                                             'WWR': w,
                                             'Insulation': th,
                                      
                                       })
samples = pd.DataFrame.from_dict(building_params)
samples

,Orientation,WWR,Insulation
0,180,0.05,0.000100
1,180,0.05,0.038978
2,180,0.05,0.077856
3,180,0.05,0.116733
4,180,0.05,0.155611
5,180,0.05,0.194489
6,180,0.05,0.233367
7,180,0.05,0.272244
8,180,0.05,0.311122
9,180,0.05,0.350000


## EnergyPlus simulation

In [5]:
# Build the dataframe where we save the results

results = pd.core.frame.DataFrame()
nameFile='/home/ict4bd/py3/project/idf_files/triple_on_.idf'

# Extract the building object from the idf file
building = ef.get_building(nameFile)
 

# Change the insulation and orientation of the building
insulation = FieldSelector(class_name='Material', object_name='XPS Extruded Polystyrene  - CO2 Blowing_.O795', field_name='Thickness')
    
orientation = FieldSelector(class_name='Building', field_name='North Axis')

# Setup the parameters
insulationPR = Parameter(selector=insulation, value_descriptor=RangeParameter(0.0001, 0.35),
                             name='Insulation')      
window_to_wall = wwr(CategoryParameter(options=[.05,.15,.5]))

orientationPR = Parameter(selector=orientation, value_descriptor=CategoryParameter(options=[180, 270]),
                          name='Orientation')

parameters = [orientationPR, window_to_wall, insulationPR]

objectives = ['Electricity:Facility', 'DistrictHeating:Facility', 'DistrictCooling:Facility']

problem = EPProblem(parameters, objectives)

/home/ict4bd/py3/lib/python3.8/site-packages/besos-2.1.3-py3.8.egg/besos/parameters.py:424: FutureWarning: Use value_descriptors instead of value_descriptor.
  warnings.warn(
/home/ict4bd/py3/lib/python3.8/site-packages/besos-2.1.3-py3.8.egg/besos/parameters.py:617: UserWarning: wwr is intended to be used with RangeParameter. Your value_descriptor is CategoryParameter(options=[0.05, 0.15, 0.5])
  warnings.warn(


In [6]:
# This evaluator uses a Problem to modify a building, and then simulate it.
# It keeps track of the building and the weather file.
evaluator = EvaluatorEP(problem, building, out_dir='outputdir_triple_on', 
                        #error_mode='Silent', 
                        err_dir='outputdir_triple_on',
                        epw='/home/ict4bd/py3/project/idf_files/GRC_Athens.167160_IWEC.epw')



In [7]:

outputs = evaluator.df_apply(samples, keep_input=True)
#Applies this evaluator to an entire dataFrame, row by row.

# :param df: a DataFrame where each row represents valid input values for this Evaluator. 
#    in this case the inputs are all the possible values of the params to be optimized
# :param keep_input: whether to include the input data in the returned DataFrame
# :param processes: amount of cores to use (only relevant when using multiprocessing)
# :param keep_dirs: whether or not keep output directory
# :return: Returns a DataFrame with one column containing the results for each objective.
        

results = results.append(outputs)
print(results)

Executing:   0%|          | 0/60 [00:00<?, ?row/s]

EnergyPlus Starting
EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2021.08.06 10:04
Initializing Response Factors
Calculating CTFs for "PAVIMENTO INTERNO DI PROGETTO", Construction # 3
Calculating CTFs for "TETTO PIANO DI PROGETTO", Construction # 5
Calculating CTFs for "PARTIZIONE DI PROGETTO", Construction # 7
Calculating CTFs for "PARTIZIONE DI PROGETTO_REV", Construction # 8
Calculating CTFs for "PARETE DI PROGETTO", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Fa

EnergyPlus Completed Successfully.


EnergyPlus Starting
EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2021.08.06 10:05
Initializing Response Factors
Calculating CTFs for "PAVIMENTO INTERNO DI PROGETTO", Construction # 3
Calculating CTFs for "TETTO PIANO DI PROGETTO", Construction # 5
Calculating CTFs for "PARTIZIONE DI PROGETTO", Construction # 7
Calculating CTFs for "PARTIZIONE DI PROGETTO_REV", Construction # 8
Calculating CTFs for "PARETE DI PROGETTO", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Fa

EnergyPlus Completed Successfully.


...

EnergyPlus Starting
EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2021.08.06 10:29
Initializing Response Factors
Calculating CTFs for "PAVIMENTO INTERNO DI PROGETTO", Construction # 3
Calculating CTFs for "TETTO PIANO DI PROGETTO", Construction # 5
Calculating CTFs for "PARTIZIONE DI PROGETTO", Construction # 7
Calculating CTFs for "PARTIZIONE DI PROGETTO_REV", Construction # 8
Calculating CTFs for "PARETE DI PROGETTO", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Fa

EnergyPlus Completed Successfully.


EnergyPlus Starting
EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2021.08.06 10:30
Initializing Response Factors
Calculating CTFs for "PAVIMENTO INTERNO DI PROGETTO", Construction # 3
Calculating CTFs for "TETTO PIANO DI PROGETTO", Construction # 5
Calculating CTFs for "PARTIZIONE DI PROGETTO", Construction # 7
Calculating CTFs for "PARTIZIONE DI PROGETTO_REV", Construction # 8
Calculating CTFs for "PARETE DI PROGETTO", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Fa

EnergyPlus Completed Successfully.


    Orientation   WWR  Insulation  Electricity:Facility  \
0           180  0.05    0.000100          7.037873e+09   
1           180  0.05    0.038978          7.037873e+09   
2           180  0.05    0.077856          7.037873e+09   
3           180  0.05    0.116733          7.037873e+09   
4           180  0.05    0.155611          7.037873e+09   
5           180  0.05    0.194489          7.037873e+09   
6           180  0.05    0.233367          7.037873e+09   
7           180  0.05    0.272244          7.037873e+09   
8           180  0.05    0.311122          7.037873e+09   
9           180  0.05    0.350000          7.037873e+09   
10          180  0.15    0.000100          7.011907e+09   
11          180  0.15    0.038978          7.011907e+09   
12          180  0.15    0.077856          7.011907e+09   
13          180  0.15    0.116733          7.011907e+09   
14          180  0.15    0.155611          7.011907e+09   
15          180  0.15    0.194489          7.011907e+09 

## Consumption

In [8]:
# Compute total area
total_area= 0
for Zones in building.idfobjects["Zone"]:
    total_area += float(Zones.checkrange("Floor_Area")) 
total_area

46.8578

In [9]:
#Consumption by hour

results['Electricity:Facility'] /= 3.6e6
results['DistrictHeating:Facility'] /= 3.6e6
results['DistrictCooling:Facility'] /= 3.6e6
results['TotalConsumption'] = results['DistrictHeating:Facility'] + results['DistrictCooling:Facility']
results

,Orientation,WWR,Insulation,Electricity:Facility,DistrictHeating:Facility,DistrictCooling:Facility,TotalConsumption
0,180,0.05,0.000100,1954.964858,3398.163088,2522.677470,5920.840558
1,180,0.05,0.038978,1954.964858,2631.491852,1646.097320,4277.589172
2,180,0.05,0.077856,1954.964858,2455.649439,1463.551288,3919.200726
3,180,0.05,0.116733,1954.964858,2377.115634,1382.152728,3759.268362
4,180,0.05,0.155611,1954.964858,2332.507429,1335.086920,3667.594349
5,180,0.05,0.194489,1954.964858,2304.189179,1306.819792,3611.008972
6,180,0.05,0.233367,1954.964858,2284.608445,1285.864183,3570.472628
7,180,0.05,0.272244,1954.964858,2269.951637,1270.666885,3540.618522
8,180,0.05,0.311122,1954.964858,2258.850043,1258.646250,3517.496292
9,180,0.05,0.350000,1954.964858,2249.956537,1249.688328,3499.644865


In [10]:
results = results.reset_index()
results


,index,Orientation,WWR,Insulation,Electricity:Facility,DistrictHeating:Facility,DistrictCooling:Facility,TotalConsumption
0,0,180,0.05,0.000100,1954.964858,3398.163088,2522.677470,5920.840558
1,1,180,0.05,0.038978,1954.964858,2631.491852,1646.097320,4277.589172
2,2,180,0.05,0.077856,1954.964858,2455.649439,1463.551288,3919.200726
3,3,180,0.05,0.116733,1954.964858,2377.115634,1382.152728,3759.268362
4,4,180,0.05,0.155611,1954.964858,2332.507429,1335.086920,3667.594349
5,5,180,0.05,0.194489,1954.964858,2304.189179,1306.819792,3611.008972
6,6,180,0.05,0.233367,1954.964858,2284.608445,1285.864183,3570.472628
7,7,180,0.05,0.272244,1954.964858,2269.951637,1270.666885,3540.618522
8,8,180,0.05,0.311122,1954.964858,2258.850043,1258.646250,3517.496292
9,9,180,0.05,0.350000,1954.964858,2249.956537,1249.688328,3499.644865


In [11]:
# Hourly consumption by square meter
results['Electricity:Facility'] /= total_area
results['DistrictHeating:Facility'] /= total_area
results['DistrictCooling:Facility'] /= total_area
results['TotalConsumption'] /= total_area
results

,index,Orientation,WWR,Insulation,Electricity:Facility,DistrictHeating:Facility,DistrictCooling:Facility,TotalConsumption
0,0,180,0.05,0.000100,41.721226,72.520756,53.836874,126.357630
1,1,180,0.05,0.038978,41.721226,56.159099,35.129633,91.288733
2,2,180,0.05,0.077856,41.721226,52.406418,31.233888,83.640306
3,3,180,0.05,0.116733,41.721226,50.730415,29.496748,80.227163
4,4,180,0.05,0.155611,41.721226,49.778424,28.492309,78.270733
5,5,180,0.05,0.194489,41.721226,49.174079,27.889056,77.063135
6,6,180,0.05,0.233367,41.721226,48.756204,27.441839,76.198042
7,7,180,0.05,0.272244,41.721226,48.443410,27.117511,75.560921
8,8,180,0.05,0.311122,41.721226,48.206489,26.860976,75.067466
9,9,180,0.05,0.350000,41.721226,48.016692,26.669804,74.686495


In [12]:
samples_for_plot = copy.deepcopy(results)
samples_for_plot = samples_for_plot.drop(columns=['DistrictHeating:Facility', 'DistrictCooling:Facility','TotalConsumption','Electricity:Facility','index'])
samples_for_plot

,Orientation,WWR,Insulation
0,180,0.05,0.000100
1,180,0.05,0.038978
2,180,0.05,0.077856
3,180,0.05,0.116733
4,180,0.05,0.155611
5,180,0.05,0.194489
6,180,0.05,0.233367
7,180,0.05,0.272244
8,180,0.05,0.311122
9,180,0.05,0.350000


## Find best settings (i.e. minimizing consumption)

In [13]:
# Best settings
min_value_index = results[['TotalConsumption']].idxmin()
print(results.iloc[min_value_index])

   index  Orientation   WWR  Insulation  Electricity:Facility  \
9      9          180  0.05        0.35             41.721226   

   DistrictHeating:Facility  DistrictCooling:Facility  TotalConsumption  
9                 48.016692                 26.669804         74.686495  


In [14]:
# Save best settings in a csv file

results["name"]=nameFile.split("/")[-1]

if HD==0:
    results.iloc[min_value_index].to_csv('opt_results.csv', mode='a', header=True)
else:
    results.iloc[min_value_index].to_csv('opt_results.csv', mode='a', header=False)

HD=1